In [1]:
import pandas as pd
import numpy as np

In [2]:
datos = pd.read_csv('C:\master\dataset.csv')

In [3]:
datos['date'] = pd.to_datetime(datos['date'])

In [4]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(datos["Shift"])
list(LE.classes_)

['Afternoon', 'Morning', 'Night']

In [5]:
datos["Shift"] = LE.transform(datos["Shift"])
datos["Shift"].value_counts()

0    279847
1    277391
2    267349
Name: Shift, dtype: int64

In [6]:
datos.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid
0,1,2010-10-08,2,0,2.08,41.41
1,1,2011-02-16,1,0,2.08,41.41
2,1,2014-05-31,2,0,2.08,41.41
3,1,2011-04-03,0,0,2.08,41.41
4,1,2013-02-20,1,0,2.08,41.41


In [7]:
from datetime import datetime, date, time, timedelta
import calendar

In [8]:
Week_Day = []
for fecha in datos['date']:
    dia_semana = datetime.weekday(fecha)+1
    Week_Day.append(dia_semana)
    
datos['Week.Day'] = Week_Day
del Week_Day

In [9]:
datos["Type.Day"] = datos["Week.Day"].apply(lambda x: 'workday' if x<6 else 'weekend')

In [10]:
datos.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid,Week.Day,Type.Day
0,1,2010-10-08,2,0,2.08,41.41,5,workday
1,1,2011-02-16,1,0,2.08,41.41,3,workday
2,1,2014-05-31,2,0,2.08,41.41,6,weekend
3,1,2011-04-03,0,0,2.08,41.41,7,weekend
4,1,2013-02-20,1,0,2.08,41.41,3,workday


In [11]:
LE.fit(datos["Type.Day"])
list(LE.classes_)

['weekend', 'workday']

In [12]:
datos["Type.Day"] = LE.transform(datos["Type.Day"])
datos["Type.Day"].value_counts()

1    591925
0    232662
Name: Type.Day, dtype: int64

In [13]:
festivos = ('1-1','6-1','19-4','22-4','1-05','10-6','24-6','15-8','11-9','24-9','12-10','1-11','6-12','8-12','25-12','26-12')

festive = []
for fecha in datos['date']:
    pasat = False
    for festivo in festivos:
        aux = festivo.split("-")
        aux2 = datetime.strftime(fecha, '%Y-%m-%d').split("-")
        if ( (aux2[1] == aux[1]) and (aux2[2] == aux[0]) and (pasat == False) ):
            festive.append(1)
            pasat = True
    if (pasat == False):
        festive.append(0)

datos['Festive'] = festive
del festive
del festivos

datos.loc[datos['Week.Day'] > 6, 'Festive'] = 1 

In [14]:
datos['Festive'].value_counts()

0    703431
1    121156
Name: Festive, dtype: int64

In [15]:
datos.head()

,GridID,date,Shift,Accident,Longitude.grid,Latitude.grid,Week.Day,Type.Day,Festive
0,1,2010-10-08,2,0,2.08,41.41,5,1,0
1,1,2011-02-16,1,0,2.08,41.41,3,1,0
2,1,2014-05-31,2,0,2.08,41.41,6,0,0
3,1,2011-04-03,0,0,2.08,41.41,7,0,1
4,1,2013-02-20,1,0,2.08,41.41,3,1,0


Árbol decisiones

In [16]:
X = datos.drop("Accident",1)
X = X.drop("date",1)

In [17]:
y = datos["Accident"]

In [18]:
print(X.shape)
print(y.shape)

(824587, 7)
(824587,)


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=1)

In [20]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(577210, 7)
(247377, 7)
(577210,)
(247377,)


In [21]:
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(criterion='entropy', random_state=1)

In [22]:
DTC.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [23]:
y_train_pred = DTC.predict_proba(X_train)

In [24]:
from sklearn.metrics import roc_auc_score

In [25]:
print(roc_auc_score(y_train, y_train_pred[:, 1]), "Árbol de decisión - Datos de entrenamiento")

0.8842975712086046 Árbol de decisión - Datos de entrenamiento


In [26]:
y_test_pred = DTC.predict_proba(X_test)

In [27]:
print(roc_auc_score(y_test, y_test_pred[:, 1]), "Árbol de decisión - Datos de test")

0.7319843960335785 Árbol de decisión - Datos de test


# Escalamos

Para ayudar al algoritmo, miramos de estandariar el conjunto de datos, hemos de estandarizar los datos "unicos" a las caracteriticas que hemos extraido a partir de estos.

In [28]:
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(X)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=15)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
LR = LogisticRegression()

In [31]:
LR.fit(X_train, y_train)
y_train_pred = LR.predict(X_train)
y_test_pred = LR.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [32]:
print(accuracy_score(y_train, y_train_pred), " Accuracy - Datos de entrenamiento")

0.955531256063644  Accuracy - Datos de entrenamiento


In [33]:
print(accuracy_score(y_test, y_test_pred), "Accuracy - Datos de test")

0.9559901162702563 Accuracy - Datos de test
